In [22]:
import sys
sys.path.append('C:\ironhack\Proyecto_final') # Hablar con el jefe
from src.funciones import scraping_colocador
import pandas as pd

In [23]:
resultado = scraping_colocador('https://www.rfevb.com/')

Scraping exitoso


In [24]:
resultado[15:30]

['2',
 'Cavanna Maximiliano',
 '(CV Guaguas)',
 '2',
 '4',
 '21',
 '0',
 '3',
 '3',
 '4',
 '3',
 '34',
 '0,4412',
 '3',
 'Bertassoni Paulo Renan']

In [25]:
resultado = [resultado[i:i+13] for i in range(3, len(resultado), 13)]

colocador = pd.DataFrame(resultado)

'''Cuidado con la modificación de columnas, drops y demas. puede cambiar.'''
    
column_names = ['Nombre', 1, 'Partidos_jugados', 'Sets_jugados', 'Acciones_exitosas', 'Errores_colocador',
                '_error_colocador', 'Puntos_negativos', 'Puntos_positivos',
                'Acciones_positivas', 'Total_acumulado', 'Efic_Ranking',2]

colocador.columns = column_names
colocador.drop(1, inplace=True, axis=1)
colocador.drop(2, inplace=True, axis=1)
colocador.drop(15, inplace=True)


In [26]:
colocador

,Nombre,Partidos_jugados,Sets_jugados,Acciones_exitosas,Errores_colocador,_error_colocador,Puntos_negativos,Puntos_positivos,Acciones_positivas,Total_acumulado,Efic_Ranking
0,,1,1,5,0,2,1,0,0,8,"0,5000"
1,Cavanna Maximiliano,2,4,21,0,3,3,4,3,34,"0,4412"
2,Bertassoni Paulo Renan,9,29,166,2,13,39,19,15,254,"0,4409"
3,Riganti Juan Martín,9,31,199,0,25,46,63,19,352,"0,3807"
4,Roberts Ignacio,8,32,216,2,26,45,32,37,358,"0,3743"
5,Lorente López Lucas,8,30,181,7,19,56,37,13,313,"0,3578"
6,Martín Pérez César,8,27,158,4,30,50,30,24,296,"0,2838"
7,Loeches Buitrago Guillermo,9,38,215,5,21,77,34,30,382,"0,2827"
8,Arjones Ferreiros Jaime,7,28,161,3,24,38,74,30,330,"0,2818"
9,Lorente Camacho Rubén,9,31,153,6,27,47,32,26,291,"0,2749"


In [29]:
colocador.drop(0, inplace=True)

In [30]:
#Extraccion de jugadores.
df= pd.read_csv('..\..\data\Jugadores.csv')
df

,Unnamed: 0,Dorsal,Nombre,Posición,Año_de_nacimiento,id_jugador,id_equipo,Edad
0,0,2,Paradela Cortón Miguel,Líbero,2004,1,1,19
1,1,3,Bouza Ferreiro Víctor,Receptor,1990,2,1,33
2,2,4,Linares José Luis,Opuesto,1998,3,1,25
3,3,5,Barrasa Portomeñe Samuel,Líbero,2007,4,1,16
4,4,6,Edward Batalla Dexter,Colocador/a,2000,5,1,23
...,...,...,...,...,...,...,...,...
163,163,15,Marí Domene Álvaro,Middle-blocker,2002,164,12,21
164,164,17,Martínez Gómez Sergio,Opuesto,2002,165,12,21
165,165,20,Hernández Robeff Tomás Agustín,Receptor,1998,166,12,25
166,166,23,Jiménez Gallego Carlos,Receptor,1995,167,12,28


In [31]:
colocador = pd.merge(colocador, df[['Nombre', 'id_jugador']], on='Nombre', how='left')

colocador


,Nombre,Partidos_jugados,Sets_jugados,Acciones_exitosas,Errores_colocador,_error_colocador,Puntos_negativos,Puntos_positivos,Acciones_positivas,Total_acumulado,Efic_Ranking,id_jugador_x,id_jugador_y
0,Cavanna Maximiliano,2,4,21,0,3,3,4,3,34,"0,4412",65.0,65
1,Bertassoni Paulo Renan,9,29,166,2,13,39,19,15,254,"0,4409",133.0,133
2,Riganti Juan Martín,9,31,199,0,25,46,63,19,352,"0,3807",70.0,70
3,Roberts Ignacio,8,32,216,2,26,45,32,37,358,"0,3743",23.0,23
4,Lorente López Lucas,8,30,181,7,19,56,37,13,313,"0,3578",97.0,97
5,Martín Pérez César,8,27,158,4,30,50,30,24,296,"0,2838",51.0,51
6,Loeches Buitrago Guillermo,9,38,215,5,21,77,34,30,382,"0,2827",88.0,88
7,Arjones Ferreiros Jaime,7,28,161,3,24,38,74,30,330,"0,2818",39.0,39
8,Lorente Camacho Rubén,9,31,153,6,27,47,32,26,291,"0,2749",116.0,116
9,Pont Perello Guillem,7,15,51,0,6,22,13,4,96,"0,2604",122.0,122


In [32]:
colocador.to_csv('..\..\data/jugadores/colocador.csv', index=False)